In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.debugger import Tracer # debugging
from IPython.display import clear_output
import ipywidgets as widgets
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import sys; sys.path.append('..')
import optimisation as op

In [ ]:
# make deterministic
np.random.seed(42)

Function to optimize:

In [ ]:
f = lambda x: np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1)
xs = np.linspace(-2, 10, num=200)

ys = f(xs)
best_y = np.max(ys)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(xs, ys, 'g-')
plt.margins(0, 0.1)
plt.xlabel('x')
plt.ylabel('cost')
plt.show()

In [ ]:
ranges = {
    'x': xs,
}
gp_params = dict(
    alpha = 1e-10, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.Matern(),
    n_restarts_optimizer = 10,
    copy_X_train = True # make a copy of the training data
)
strategy = op.AcquisitionStrategy(
    pre_phase_steps = 2,
    acquisition_function = ('UCB', {'kappa' : 5}),
    #parallel_strategy = 'KB'
    parallel_strategy = ('MC', {'N' : 20, 'sim_train_its' : 0})
)


In [ ]:
class MyEvaluator(op.Evaluator):
    def __init__(self, delay):
        super(MyEvaluator, self).__init__()
        self.delay = delay
    def test_config(self, config):
        time.sleep(self.delay)
        return f(config.x)

In [ ]:
optimiser = op.BayesianOptimisationOptimiser(ranges, maximise_cost=True, acquisition_strategy=strategy,
                                             gp_params=gp_params, close_tolerance=1e-3)

In [ ]:
op.gui.optimiser_progress_bar(optimiser)
#evaluators = [MyEvaluator(0.7), MyEvaluator(0.3), MyEvaluator(0.5)]
evaluators = [MyEvaluator(1), MyEvaluator(1.5), MyEvaluator(1.3)]
optimiser.run_multithreaded(evaluators, max_jobs=15)

In [ ]:
optimiser.plot_cost_over_time(true_best=best_y);

In [ ]:
def plot(n, step):
    optimiser.plot_step_1D('x', n, true_cost=f);
op.gui.step_log_slider(optimiser, plot, pre_compute=False);

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=True)
ra.run_sequential(MyEvaluator(0), max_jobs=20)

In [ ]:
ra.plot_cost_over_time(true_best=best_y);

# Plot the samples to compare with the Bayesian samples

In [ ]:
r_xs = [s.config.x for s in ra.samples]
r_ys = [s.cost for s in ra.samples]

plt.figure(figsize=(16,6))
plt.plot(xs, ys, 'g-')
plt.plot(r_xs, r_ys, 'bo', zorder=10)
plt.margins(0.1, 0.1)
plt.xlabel('x')
plt.ylabel('cost')
plt.show()